In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

We started by analyzing the html page and locating the data that we need i.e the ranking, country and region, total and international academic stuff, as well as total and international number of students. As mentioned in the homework description the information about the number of staff and students is located in a page other than the one that provides the rank ordering. A redirection to that page is provided by a link on the name of the university and is provided by the root/default/home url suffixed with "/university/some-format-of-the-university-name". Our next task is finding a way to collect the data. 

In order to discover the way the webpage fetches the data we used postman with its interceptor extension, which enabeled us to monitor the requests generated upon every navigation to the given web page. In particular we learned that the data in our interest is returned from an API with a get request to the *data_url* (used in the cell below) in json format. After inspecting the response json object, it has been noticed that all the data displayed on the rankings page originates precisely from this source, and is available in a convinient format. Even more a link to the individual webpage for every university is available, which gives us a way to collect the rest of the data we need. That data though has to be collected by crawling the html response.

In [40]:
# the home URL used for generating the URLs to every individual university site
root_url = 'http://www.topuniversities.com'
# the URL to the API that returns the university data used to generate the table/list of universities (mentioned above)
data_url = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508229310180'
data_request = requests.get(data_url)

In [44]:
# parsing the data and extracting the list of universities from the json
data_json = data_request.json()
ordered_list_of_universities = data_json['data']

Now as we have all that we have all of the data at reach, we just parse it in the following manner. 

First we extract the data available directly in the dictionay such as title, country, region and rank. The rank in particular with using a regex is trimmed from all of the possible nondigit signes present in it, in order to be succesfully parsed as an int as is should be (this is a more general solution that will work in wider spectrum of problems, while our motivation is eliminating the '=' symbols present to indicate a situation where more than one university is given the same rank i.e its a draw).

Next, we use the URL in the dictionary that leads to the individual website for the university, which in turn after crawling it's HTML provides us with the rest of the information we need. After inspection, the nodes in the DOM that contain the numbers of interest are located, and using given CSS selectors manipulated and extracted.

In [5]:
i = 0
university_parsed_data = []
for uni_data in ordered_list_of_universities:
    # This try is present in order to catch any potentional error while parsing and print it nicely
    try: 
        uni_name = uni_data['title']
        uni_country = uni_data['country']
        uni_region = uni_data['region']
        uni_rank = int(re.sub(r'^[^\d]+|[^\d]+$', '', uni_data['rank_display']))
        uni_url = uni_data['url']

        uni_url_request = requests.get("%s%s" % (root_url, uni_url))
        soup = BeautifulSoup(uni_url_request.text, 'html.parser')

        # Usually two objects are selected, since the website has mobile view and desktop view of the table
        # For some universities this information is not available, hence we save the value as -1 to note that
        # In the selection it is noted that all of the numbers of interest are contained in a div with class number, that
        # in turn is always contained in another parent div whose class somehow informs us about its semantical meaning
        # We use the classes and node types in CSS selectors to get the information of interest.
        # The information is trimmed of white spaces and transformed in a suitable format
        total_afs = soup.select('div.total.faculty div.number')
        if len(total_afs) == 0:
            total_afs = -1
        else:
            total_afs = total_afs[0].text.strip().replace(',', '') # total academic faculty stuff
            
        international_afs = soup.select('div.inter.faculty div.number')
        if len(international_afs) == 0:
            international_afs = -1
        else:
            international_afs = international_afs[0].text.strip().replace(',', '') # international academic faculty staff number
        
        total_students = soup.select('div.total.student div.number')
        if len(total_students) == 0:
            total_students = -1
        else:
            total_students = total_students[0].text.strip().replace(',', '') # total number of students

        inter_students = soup.select('div.total.inter div.number')
        if len(inter_students) == 0:
            inter_students = -1
        else:
            inter_students = inter_students[0].text.strip().replace(',', '') # international students number
        
        # Combine all of the collected data in a concise dictionary, and append it to a list later used to create a dataframe
        university_parsed_data.append({'Rank': uni_rank, 'Name': uni_name, 'Country': uni_country,  'Region': uni_region,
                        'Total_AFS': int(total_afs), 'International_AFS': int(international_afs), 
                        'Total_students': total_students, 'International_students': inter_students})
    except ValueError as e:
        print(e)
        print("ERROR")
        break
    i += 1
    if i == 200:
        print("SUCCESS")
        break

SUCCESS


In [7]:
# NaN values are set to -1
# There is no university ranked 198, in particular the ranking goes from 3 universities ranked 195 and then continues to 199 
data = pd.DataFrame(university_parsed_data)[['Rank', 'Name', 'Country', 'Region', 
                        'Total_AFS', 'International_AFS', 
                        'Total_students', 'International_students']]
data['Total_students'] = data.Total_students.astype(np.int64)
data['International_students'] = data.International_students.astype(np.int64)
data['Total_AFS'] = data.Total_AFS.astype(np.int64)
data['International_AFS'] = data.International_AFS.astype(np.int64)
data.head()

,Rank,Name,Country,Region,Total_AFS,International_AFS,Total_students,International_students
0,1,Massachusetts Institute of Technology (MIT),United States,North America,2982,1679,11067,3717
1,2,Stanford University,United States,North America,4285,2042,15878,3611
2,3,Harvard University,United States,North America,4350,1311,22429,5266
3,4,California Institute of Technology (Caltech),United States,North America,953,350,2255,647
4,5,University of Cambridge,United Kingdom,Europe,5490,2278,18770,6699


Since it is noticed that some of the part is repetative, a function is defined in order to provent code repetition.

This function gets a *DataFrame* and two *column names*, removes all the rows that contain none in either of them, and than calculates the ratio between the values in these columns for each row of the dataframe, while firstly grouping the data of the dataframe by a given column if a column of the dataframe is passed to the call to group by. The returning DataFrame is sorted by the value of the calculated ratio, which is joined to every row in a column passed as the parameter *new_column_name*. 

It is assumed that the passed *DataFrame* has columns anmed *col1*, *col2* and *group_by* (if used), as well as that the values for *col1*, *col2* are numberic.

In [8]:
def find_ratio(df, col1, col2, new_column_name, group_by=None):
    no_nan_in_columns_df = df[(df[col1] != -1) & (df[col2] != -1)]
    if group_by != None:
        no_nan_in_columns_df = no_nan_in_columns_df.groupby(group_by)[[col1, col2]].sum()
    sorted_ratio = (no_nan_in_columns_df[col1] / no_nan_in_columns_df[col2]).sort_values(ascending=False)
    return no_nan_in_columns_df.loc[sorted_ratio.index].join(pd.DataFrame(sorted_ratio, columns=[new_column_name]))

In [12]:
# Calculate the ratio between the number of staff members and students, sort the dataframe according to that value
# It is assumed that the higher the ratio in favor of the staff members the better
find_ratio(data, 'Total_AFS', 'Total_students', 'Ratio').head(10)

,Rank,Name,Country,Region,Total_AFS,International_AFS,Total_students,International_students,Ratio
3,4,California Institute of Technology (Caltech),United States,North America,953,350,2255,647,0.422616
15,16,Yale University,United States,North America,4940,1708,12402,2469,0.398323
5,6,University of Oxford,United Kingdom,Europe,6750,2964,19720,7353,0.342292
4,5,University of Cambridge,United Kingdom,Europe,5490,2278,18770,6699,0.292488
16,17,Johns Hopkins University,United States,North America,4462,1061,16146,4105,0.276353
1,2,Stanford University,United States,North America,4285,2042,15878,3611,0.269870
0,1,Massachusetts Institute of Technology (MIT),United States,North America,2982,1679,11067,3717,0.269450
185,186,University of Rochester,United States,North America,2569,488,9636,2805,0.266604
18,19,University of Pennsylvania,United States,North America,5499,1383,20639,4250,0.266437
17,18,Columbia University,United States,North America,6189,913,25045,8105,0.247115


In [19]:
# Calculate the ratio between the number of staff international students and  total number of students, sort the dataframe according to that value
# It is assumed that we are interested in the universities that have higher number of international students
find_ratio(data, 'International_students', 'Total_students', 'Ratio').head(10)

,Rank,Name,Country,Region,Total_AFS,International_AFS,Total_students,International_students,Ratio
34,35,London School of Economics and Political Scien...,United Kingdom,Europe,1088,687,9760,6748,0.691393
11,12,Ecole Polytechnique Fédérale de Lausanne (EPFL),Switzerland,Europe,1695,1300,10343,5896,0.570047
7,8,Imperial College London,United Kingdom,Europe,3930,2071,16090,8746,0.543567
198,200,Maastricht University,Netherlands,Europe,1277,502,16385,8234,0.502533
47,47,Carnegie Mellon University,United States,North America,1342,425,13356,6385,0.478062
6,7,UCL (University College London),United Kingdom,Europe,6345,2554,31080,14854,0.477928
91,92,University of St Andrews,United Kingdom,Europe,1140,485,8800,4030,0.457955
41,41,The University of Melbourne,Australia,Oceania,3311,1477,42182,18030,0.427434
126,127,Queen Mary University of London,United Kingdom,Europe,1885,801,16135,6806,0.421816
25,26,The University of Hong Kong,Hong Kong,Asia,3012,2085,20214,8230,0.407144


In [36]:
# Group the data according to the value of the country attribute
# Calculate the ratio between the number of staff members and students, sort the dataframe according to that value
find_ratio(data, 'Total_AFS', 'Total_students', 'Ratio', 'Country').head()

,Total_AFS,Total_students,Ratio
Country,,,
Russia,6709,30233,0.221910
Denmark,11916,67223,0.177261
Saudi Arabia,1062,6040,0.175828
Singapore,9444,58466,0.161530
Malaysia,2755,17902,0.153893


In [35]:
# Group the data according to the value of the country attribute
# Calculate the ratio between the number of staff international students and  total number of students, sort the dataframe according to that value
find_ratio(data, 'International_students', 'Total_students', 'Ratio', 'Country').head()

,International_students,Total_students,Ratio
Country,,,
Australia,106359,301994,0.352189
United Kingdom,199426,583621,0.341705
Hong Kong,24499,78838,0.310751
Austria,19667,63446,0.309980
Switzerland,32995,109112,0.302396


In [30]:
# Group the data according to the value of the region attribute
# Calculate the ratio between the number of staff members and students, sort the dataframe according to that value
find_ratio(data, 'Total_AFS', 'Total_students', 'Ratio', 'Region').head()

,Total_AFS,Total_students,Ratio
Region,,,
Asia,106734,807003,0.132260
North America,182123,1546353,0.117776
Europe,218358,1957251,0.111564
Latin America,45382,435750,0.104147
Africa,1733,19593,0.088450


In [34]:
# Group the data according to the value of the region attribute
# Calculate the ratio between the number of staff international students and  total number of students, sort the dataframe according to that value
find_ratio(data, 'International_students', 'Total_students', 'Ratio', 'Region').head()

,International_students,Total_students,Ratio
Region,,,
Oceania,118798,350167,0.339261
Europe,449364,1957251,0.229589
North America,292116,1546353,0.188906
Africa,3325,19593,0.169703
Asia,110100,807003,0.136431
